In [ ]:
# Research Scoping Agent Implementation
# This notebook implements the research scoping agent as described in the guide.

# Import required libraries
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, AIMessage, get_buffer_string
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver
from pydantic import BaseModel, Field
from research_agent_scope import clarify_with_user, write_research_brief
from state_scope import AgentState, AgentInputState
from prompts import clarify_with_user_instructions, transform_messages_into_research_topic_prompt
import operator
from typing import Optional, Sequence, Annotated
from langchain_core.messages import BaseMessage
from datetime import datetime
from langgraph.types import Command
from typing import Literal

In [ ]:
# Build the scoping workflow
deep_researcher_builder = StateGraph(AgentState, input_schema=AgentInputState)

# Add workflow nodes
deep_researcher_builder.add_node("clarify_with_user", clarify_with_user)
deep_researcher_builder.add_node("write_research_brief", write_research_brief)

# Add workflow edges
deep_researcher_builder.add_edge(START, "clarify_with_user")
deep_researcher_builder.add_edge("write_research_brief", END)

# Compile the workflow
scope_research = deep_researcher_builder.compile()

In [ ]:
# Compile with in-memory checkpointer for testing
checkpointer = InMemorySaver()
scope = deep_researcher_builder.compile(checkpointer=checkpointer)

In [ ]:
# Visualize the graph
from IPython.display import Image, display
display(Image(scope.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
# Test with a sample query
result = scope.invoke({
    "messages": [HumanMessage(content="I want to research the best coffee shops in San Francisco.")]
})

print("Result:", result)